In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=bb4707b397ea23c305dc31cbd2b74c389fdf6e4deb04b9a9f2d3813b0ca97367
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 4.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F15 '''
def F15(X):
    f = bn.F15()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.32 ,random_state=0 , l1_ratio=0.9795, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_15_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_15_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20, 30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=395550, Sun Apr 19 13:22:59 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.008330224341845e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.894824999224801e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.780116236768600e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [23055.89005434] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [19950.2068003] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   24    456 8.663537522832943e+03 1.0e+00 2.05e+00  2e+00  2e+00 0:03.4
   48    912 3.831081030794663e+03 1.1e+00 1.89e+00  2e+00  2e+00 0:07.4
  100   1900 -7.872169230301406e+02 1.2e+00 1.49e+00  1e+00  2e+00 0:12.2
  167   3173 -1.389504411802078e+04 1.2e+00 8.52e-01  8e-01  9e-01 0:18.2
  200   3800 -1.937959706938439e+04 1.2e+00 6.10e-01  6e-01  6e-01 0:21.3
  289   5491 -2.813796357553182e+04 1.2e+00 2.34e-01  2e-01  2e-01 0:29.3
  300   5700 -2.843354468226152e+04 1.2e+00 2.08e-01  2e-01  2e-01 0:30.3
  400   7600 -2.959805269751958e+04 1.2e+00 7.25e-02  7e-02  7e-02 0:39.6
  500   9500 -2.975740701700351e+04 1.2e+00 2.59e-02  3e-02  3e-02 0:48.8
  600  11400 -2.977417856589295e+04 1.2e+00 8.26e-03  8e-03  8e-03 0:57.8
  700  13300 -2.977585631513775e+04 1.2e+00 2.74e-03  3e-03  3e-03 1:06.9
  800  15200 -2.977603356060832e+04 1.2e+00 9.33e-04  9e-04  9e-04 1:15.9
  900  17100 -2.977605386644905e+04 1.3e+00 2.93e-04  3e-04  3e-04 1:24.9
 1000  19000 -2.977605624644893e+04 1.3e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [20084.57006765] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [24264.81860629] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.800093979096965e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.787819427176116e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

    3     57 1.669426700387996e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   39    741 4.200785557097155e+03 1.1e+00 1.95e+00  2e+00  2e+00 0:03.3
   86   1634 9.019230726867172e+02 1.2e+00 1.49e+00  1e+00  2e+00 0:07.4
  100   1900 -9.296623516938271e+02 1.2e+00 1.35e+00  1e+00  1e+00 0:08.6
  171   3249 -1.555235233746265e+04 1.2e+00 7.58e-01  8e-01  8e-01 0:14.7
  200   3800 -2.103809113839133e+04 1.2e+00 5.71e-01  6e-01  6e-01 0:17.2
  292   5548 -2.847652333201881e+04 1.2e+00 2.02e-01  2e-01  2e-01 0:25.2
  300   5700 -2.861856927808273e+04 1.2e+00 1.86e-01  2e-01  2e-01 0:25.9
  400   7600 -2.964273024309998e+04 1.2e+00 6.54e-02  6e-02  7e-02 0:34.5
  500   9500 -2.976253465483844e+04 1.2e+00 2.22e-02  2e-02  2e-02 0:43.2
  600  11400 -2.977480657852922e+04 1.2e+00 7.01e-03  7e-03  7e-03 0:51.6
  700  13300 -2.977589960159999e+04 1.2e+00 2.40e-03  2e-03  2e-03 1:00.2
  800  15200 -2.977603873183953e+04 1.2e+00 8.07e-04  8e-04  8e-04 1:08.6
  900  17100 -2.977605464136820e+04 1.2e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [20084.57006765] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [23393.69893989] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.640160576523673e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.636082181766970e+04 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [21283.71368468] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   36    684 2.667678490444339e+03 1.1e+00 1.85e+00  2e+00  2e+00 0:03.3
   80   1520 -2.950664140880430e+03 1.1e+00 1.50e+00  1e+00  2e+00 0:07.4
  100   1900 -4.508763909068975e+03 1.2e+00 1.35e+00  1e+00  1e+00 0:09.4
  164   3116 -1.486675316842540e+04 1.2e+00 8.56e-01  8e-01  9e-01 0:15.5
  200   3800 -1.807436984563784e+04 1.2e+00 6.51e-01  6e-01  7e-01 0:18.9
  286   5434 -2.415118885169696e+04 1.2e+00 3.42e-01  3e-01  3e-01 0:26.9
  300   5700 -2.502467743485369e+04 1.2e+00 3.10e-01  3e-01  3e-01 0:28.2
  400   7600 -2.759853144908448e+04 1.3e+00 1.67e-01  2e-01  2e-01 0:37.7
  498   9462 -2.846166920921577e+04 1.3e+00 1.00e-01  1e-01  1e-01 0:48.8
  500   9500 -2.847648380143584e+04 1.3e+00 9.91e-02  1e-01  1e-01 0:49.2
  600  11400 -2.882698572914786e+04 1.5e+00 5.89e-02  6e-02  6e-02 1:01.5
  700  13300 -2.905016384575576e+04 1.7e+00 5.06e-02  5e-02  5e-02 1:10.8
  800  15200 -2.920163048588693e+04 2.1e+00 4.85e-02  5e-02  6e-02 1:20.1
  900  17100 -2.930179468212962e+04 2.4

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('sample_data\\F15_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F15(X_Values.iloc[i,:200])
    SVM_Fun [i] = F15(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F15(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([3719.67591773]), count=array([1])) 0.00011376060086498142
SVM
ModeResult(mode=array([18352.58735243]), count=array([1])) 2577.6992822284624
ELN
ModeResult(mode=array([22699.55404683]), count=array([1])) 4079.0906968851245
